In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## Define the class
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

In [8]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0946, 0.0990, 0.0978, 0.0991, 0.1073, 0.1050, 0.1003, 0.1050, 0.0957,
         0.0961]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([4])


In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [14]:
layer_1 = nn.Linear(in_features= 28*28, out_features=20)
hidden1 = layer_1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [15]:
print(f"Before ReLU: {hidden1} \n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1} \n")

Before ReLU: tensor([[ 0.1949,  0.0962, -0.4345, -0.0237, -0.4161, -0.5999,  0.0263,  0.5494,
          0.2637, -0.5219, -0.0817,  0.2755,  0.0035,  0.0845,  0.1853, -0.2283,
          0.6216,  0.0586, -0.1746, -0.2639],
        [ 0.1411,  0.1663, -0.2678, -0.5014, -0.3564,  0.0329,  0.0257,  0.4934,
          0.3611, -0.2367, -0.3254,  0.2110,  0.0509,  0.0558, -0.0629, -0.1329,
          0.7193,  0.2197,  0.2108,  0.1739],
        [-0.0047,  0.1184,  0.0908, -0.3528, -0.1847, -0.2807, -0.1551,  0.2372,
          0.7463, -0.1838,  0.1153,  0.5821,  0.1879,  0.3467,  0.2397, -0.0064,
          0.6171,  0.1340,  0.0573, -0.0485]], grad_fn=<AddmmBackward0>) 

After ReLU: tensor([[0.1949, 0.0962, 0.0000, 0.0000, 0.0000, 0.0000, 0.0263, 0.5494, 0.2637,
         0.0000, 0.0000, 0.2755, 0.0035, 0.0845, 0.1853, 0.0000, 0.6216, 0.0586,
         0.0000, 0.0000],
        [0.1411, 0.1663, 0.0000, 0.0000, 0.0000, 0.0329, 0.0257, 0.4934, 0.3611,
         0.0000, 0.0000, 0.2110, 0.0509, 0.0558, 0.00

In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer_1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.0878,  0.1558,  0.1121,  0.0042, -0.1056, -0.1544, -0.2648, -0.1318,
          0.0309, -0.0094],
        [-0.0065,  0.1106,  0.1916,  0.0031, -0.1146, -0.0050, -0.3082, -0.0694,
         -0.0225,  0.0206],
        [-0.0709,  0.1192,  0.2312,  0.0862, -0.0587, -0.1722, -0.3620, -0.1318,
          0.0342, -0.0998]], grad_fn=<AddmmBackward0>)


In [21]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0951, 0.1214, 0.1162, 0.1043, 0.0934, 0.0890, 0.0797, 0.0910, 0.1071,
         0.1029],
        [0.1006, 0.1131, 0.1226, 0.1016, 0.0903, 0.1007, 0.0744, 0.0944, 0.0990,
         0.1033],
        [0.0960, 0.1161, 0.1298, 0.1123, 0.0972, 0.0867, 0.0717, 0.0903, 0.1066,
         0.0932]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([1, 2, 2])


In [22]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0081,  0.0211,  0.0151,  ..., -0.0151,  0.0179, -0.0092],
        [ 0.0345, -0.0051, -0.0311,  ..., -0.0174,  0.0256,  0.0343]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0345, -0.0210], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0056, -0.0219, -0.0013,  ..., -0.0231,  0.0263, -0.0409],
        [-0.0364, -0.0414, -0.0121,  ...,  0.0063,  0.0069, -0.0241]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 